# Notebook 02: Optimization

This notebook demonstrates the core optimization process for placing ambulance stations in Abu Dhabi to maximize population coverage within an 8-minute response window.

## 1. Load Data & Initialize

In [ ]:
import os
import sys
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

# Add root to path to import optimization module
PROJECT_ROOT = os.path.dirname(os.getcwd())
sys.path.append(PROJECT_ROOT)

from optimization.mclp_model import run_mclp
from optimization.coverage_matrix import compute_travel_time_matrix, build_coverage_matrix
from visualization.solution_plots import plot_mip_solution

DATA_DIR = os.path.join(PROJECT_ROOT, "data", "synthetic")

## 2. Load Spatial Components

In [ ]:
zones_gdf = gpd.read_file(os.path.join(DATA_DIR, "zones.geojson"))
demand_gdf = gpd.read_file(os.path.join(DATA_DIR, "demand_nodes.geojson"))
candidates_gdf = gpd.read_file(os.path.join(DATA_DIR, "candidate_stations.geojson"))
print(f"Demand Nodes: {len(demand_gdf)}")
print(f"Candidate Sites: {len(candidates_gdf)}")

## 3. Build Matrices

We calculate the travel time between every demand center and every potential station site, then create a binary 'coverage' matrix based on the 8-minute threshold.

In [ ]:
time_matrix = compute_travel_time_matrix(demand_gdf, candidates_gdf)
cov_matrix = build_coverage_matrix(time_matrix, threshold_min=8.0)
weights = demand_gdf.weight.values

print(f"Matrix shape: {cov_matrix.shape}")

## 4. Run the Maximum Coverage Model

We use the **Maximum Coverage Location Problem (MCLP)** formulation. We have a budget of **12 stations** and want to maximize the covered population weight.

In [ ]:
model = run_mclp(cov_matrix, weights, p_stations=12, p_vehicles=24, verbose=False)
print(model.summary())

## 5. Visualize the Solution

The green circles represent the 8-minute coverage radius for each optimized station site.

In [ ]:
results = model._get_results()
plot_mip_solution(
    zones_gdf, 
    candidates_gdf, 
    results["open_stations"], 
    demand_gdf=demand_gdf
)
plt.show()